In [ ]:
import numpy as np 
import pandas as pd
import random
import os
import math
import time

In [ ]:
!pip install timm

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from PIL import Image as pil_image
from tqdm import tqdm
import scipy

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import timm
from timm.optim import Lookahead, RAdam

In [ ]:
IMG_SIZE = 512
SEED = 42
PROJECT_FOLDER = "../input/hotel-id-2022-fgvc9/"
DATA_FOLDER = "../input/hotelid-2022-train-images-512x512/"
IMAGE_FOLDER = DATA_FOLDER
OUTPUT_FOLDER = ""

train_df = pd.read_csv(os.path.join(DATA_FOLDER, 'train.csv'))

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2 


base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

train_transform = A.Compose([
    #A.HorizontalFlip(p=0.2),
    #A.ShiftScaleRotate(p=0.2, border_mode=cv2.BORDER_CONSTANT),
    #A.RandomBrightness(p=0.2),
    A.CoarseDropout(p=1., max_holes=1, 
                    min_height=IMG_SIZE//4, max_height=IMG_SIZE//2,
                    min_width=IMG_SIZE//4,  max_width=IMG_SIZE//2, 
                    fill_value=(255,0,0)), #Occlusions in test data simulation
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

valid_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])


In [ ]:
class HotelTrainDataset:
    def __init__(self, data, transform=None, data_path=""):
        self.data = data
        self.data_path = data_path
        self.data_path += "images/"
        self.transform = transform
        self.fake_load = False

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_path + record["image_id"]

        if self.fake_load:
            image = np.random.randint(0, 255, (32, 32, 3)).astype(np.uint8)
        else:
            image = np.array(pil_image.open(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
        
        return {
            "image" : transformed["image"],
            "target" : record['hotel_id_code'],
        }

In [ ]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s

        return output

In [ ]:
class HotelIdModel(nn.Module):
    def __init__(self, out_features, embed_size=256, backbone_name="efficientnet_b3"):
        super(HotelIdModel, self).__init__()

        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=True)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)

        self.arc_face = ArcMarginProduct(self.embed_size, out_features, s=64.0, m=0.50, easy_margin=False) #Changed ArcMargin parameters s=64.0 & m=0.50

        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
            nn.BatchNorm1d(self.embed_size),
        )

        print(f"Model {backbone_name} ArcMarginProduct - Features: {in_features}, Embeds: {self.embed_size}")
        
    def forward(self, input, targets = None):
        x = self.backbone(input)
        x = x.view(x.size(0), -1)
        x = self.post(x)
        
        if targets is not None:
            logits = self.arc_face(x, targets)
            return logits
        
        return x

In [ ]:
def get_embeds(loader, model, bar_desc="Generating embeds"):
    targets_all = []
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            target = sample['target'].to(args.device)
            output = model(input)

            targets_all.extend(target.cpu().numpy())
            outputs_all.extend(output.detach().cpu().numpy())
            
    return targets_all, outputs_all

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
    
def get_distance_matrix(embeds, base_embeds):
    distance_matrix = []
    embeds_dataset = torch.utils.data.TensorDataset(torch.Tensor(embeds))
    embeds_dataloader = DataLoader(embeds_dataset, num_workers=2, batch_size=1024, shuffle=False)
    
    t = tqdm(embeds_dataloader)
    for i, sample in enumerate(t): 
        distances = cosine_similarity(sample[0].numpy(), base_embeds)
        distance_matrix.extend(distances)
        
    return np.array(distance_matrix)

In [ ]:
def save_checkpoint(model, scheduler, optimizer, epoch, name, loss=None, score=None):
    checkpoint = {"epoch": epoch,
                  "model": model.state_dict(),
                  "scheduler": scheduler.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  "loss": loss,
                  "score": score,
                  }

    torch.save(checkpoint, f"{OUTPUT_FOLDER}checkpoint-{name}.pt")


def load_checkpoint(model, scheduler, optimizer, name):
    checkpoint = torch.load(f"{OUTPUT_FOLDER}checkpoint-{name}.pt")

    model.load_state_dict(checkpoint["model"])
    scheduler.load_state_dict(checkpoint["scheduler"])

    return model, scheduler, optimizer, checkpoint["epoch"]

In [ ]:
def iterate_loader(loader, epochs):
    """
    Iterates through data loader with fake load (empty data) instead
    of reading the real images to speed up. Dataloader has no state_dict
    so manual iterating is need to get the loader with augmentations to 
    correct state.
    """
    loader.dataset.fake_load = True
    with torch.no_grad():
        for i in range(epochs):
            t = tqdm(loader, desc=f"Iterating loader {i+1}/{epochs}")
            for j, sample in enumerate(t):
                images = sample['image']
                targets = sample['target']

    loader.dataset.fake_load = False

In [ ]:
def train_epoch(args, model, loader, criterion, optimizer, scheduler, epoch):
    losses = []
    targets_all = []
    outputs_all = []
    
    model.train()
    t = tqdm(loader)
    
    for i, sample in enumerate(t):
        optimizer.zero_grad()
        
        input = sample['image'].to(args.device)
        target = sample['target'].to(args.device)
        
        output = model(input, target)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()

        if scheduler is not None:
            scheduler.step()
        
        losses.append(loss.item())
        targets_all.extend(target.cpu().numpy())
        outputs_all.extend(torch.sigmoid(output).detach().cpu().numpy())
        
        score = accuracy_score(targets_all, np.argmax(outputs_all, axis=1))
        t.set_description(f"Training epoch {epoch}/{args.epochs} - Train loss:{loss:0.4f}, score: {score:0.4f}")
        
    return np.mean(losses), score

In [ ]:
def find_closest_match(base_df, distance_matrix, n_matches=5):
    preds = []
    N_dist = len(distance_matrix)
    for i in tqdm(range(N_dist), total=N_dist, desc="Getting closest match"):
        tmp_df = base_df.copy()
        tmp_df["distance"] = distance_matrix[i]
        tmp_df = tmp_df.sort_values(by=["distance", "hotel_id"], ascending=False).reset_index(drop=True)
        preds.extend([tmp_df["hotel_id_code"].unique()[:n_matches]])
    
    preds = np.array(preds)
    return preds

In [ ]:
def calc_metric(y_true, y_pred, n_matches=5):
    y = np.repeat([y_true], repeats=n_matches, axis=0).T
    acc_top_1 = (y_pred[:, 0] == y_true).mean()
    acc_top_5 = (y_pred == y).any(axis=1).mean()
    print(f"Accuracy: {acc_top_1:0.4f}, top 5 accuracy: {acc_top_5:0.4f}")
    return acc_top_1, acc_top_5

In [ ]:
def test(base_loader, valid_loader, model):
    base_targets, base_embeds = get_embeds(base_loader, model, "Generating embeds for train")
    valid_targets, valid_embeds = get_embeds(valid_loader, model, "Generating embeds for test")
    distance_matrix = get_distance_matrix(valid_embeds, base_embeds)
    val_preds = find_closest_match(base_loader.dataset.data, distance_matrix)
    calc_metric(valid_targets, val_preds)
    return base_embeds, valid_embeds, valid_targets, val_preds, distance_matrix

In [ ]:
def test_closest_match_tta(args, base_loader, valid_df, tta_transforms, model):
    base_targets, base_embeds = get_embeds(base_loader, model, "Generating embeds for train")
    distance_matrix = None

    for key in tta_transforms:
        valid_dataset = HotelTrainDataset(valid_df, tta_transforms[key], data_path=DATA_FOLDER)
        valid_loader = DataLoader(valid_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
        valid_targets, valid_embeds = get_embeds(valid_loader, model, f"Generating embeds for test {key}")
        
        distances = get_distance_matrix(valid_embeds, base_embeds)

        if distance_matrix is None:
            distance_matrix = distances
        else:
            distance_matrix = np.min(np.dstack((distance_matrix, distances)), axis = 2)
    
    val_preds = find_closest_match(base_loader.dataset.data, distance_matrix)
    calc_metric(valid_targets, val_preds)

In [ ]:
def sample_data(n_hotels, min_images, max_images):
    data_df = pd.read_csv(DATA_FOLDER + "train.csv")
    sample_df = data_df.groupby("hotel_id").filter(lambda x: (x["image_id"].nunique() > min_images) & (x["image_id"].nunique() < max_images))
    sample_df["hotel_id_code"] = sample_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)
    sample_df = sample_df[sample_df["hotel_id_code"] < n_hotels]

    print(f"Subsample with {len(sample_df.hotel_id.unique())} hotels out of {len(data_df.hotel_id.unique())} " + 
          f"with total {len(sample_df)} images ({len(sample_df) / len(data_df) * 100:0.2f} %)")
    
    return sample_df

In [ ]:
# FOR TESTING DIFFERENT SETTING
#data_df = sample_data(1000, 15, 50)

# FOR FINAL TRAINING
data_df = pd.read_csv(DATA_FOLDER + "train.csv")
data_df["hotel_id_code"] = data_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)

fig = go.Figure()
fig.add_trace(go.Histogram(x=data_df["hotel_id_code"]))
fig.update_xaxes(type="category")
fig.show()

In [ ]:
data_df.head()

In [ ]:
def train_and_validate(args, data_df):
    model_name = f"arcmargin-model-{args.backbone_name}-{IMG_SIZE}x{IMG_SIZE}-{args.embed_size}embeds-{args.n_classes}hotels"
    print(model_name)
    #SEED and split
    seed_everything(seed=SEED)
    valid_df = data_df.groupby("hotel_id").sample(args.val_samples, random_state=SEED)
    train_df = data_df[~data_df["image_id"].isin(valid_df["image_id"])]

    #Create model
    model = HotelIdModel(args.n_classes, args.embed_size, args.backbone_name)
    model = model.to(args.device)

    #Train data loader
    train_dataset = HotelTrainDataset(train_df, valid_transform, data_path=DATA_FOLDER)
    train_loader = DataLoader(train_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=True, pin_memory=False)
    #Train without augmentations to generate base embeddings
    base_dataset = HotelTrainDataset(train_df, valid_transform, data_path=DATA_FOLDER)
    base_loader = DataLoader(base_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
    #Valid loader
    valid_dataset = HotelTrainDataset(valid_df, valid_transform, data_path=DATA_FOLDER)
    valid_loader = DataLoader(valid_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

    print(f"Base: {len(base_dataset)}\nValidation: {len(valid_dataset)}")

    criterion = nn.CrossEntropyLoss()
    optimizer = Lookahead(torch.optim.AdamW(model.parameters(), lr=args.lr), k=3)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=args.lr,
                    epochs=args.epochs,
                    steps_per_epoch=len(train_loader),
                    div_factor=10,
                    final_div_factor=1,
                    pct_start=0.1,
                    anneal_strategy="cos",
                )
    
    start_epoch = 1

    if args.continue_from_checkpoint:
        model, scheduler, optimizer, last_epoch = load_checkpoint(model, scheduler, optimizer, model_name)
        iterate_loader(train_loader, last_epoch)
        start_epoch = start_epoch + last_epoch

    torch.cuda.empty_cache()

    for epoch in range(start_epoch, args.epochs +1):
        train_loss, train_score = train_epoch(args, model, train_loader, criterion, optimizer, scheduler, epoch)
        save_checkpoint(model, scheduler, optimizer, epoch, model_name, train_loss, train_score)
        if (epoch == 1):
            base_embeds, valid_embeds, valid_targets, val_preds, distance_matrix = test(base_loader, valid_loader, model)

    base_embeds, valid_embeds, valid_targets, val_preds, distance_matrix = test(base_loader, valid_loader, model)

    output = {"base_embeds": base_embeds,
              "valid_embeds": valid_embeds,
              "valid_targets": valid_targets,
              "val_preds": val_preds,
              "distance_matrix": distance_matrix,
              "train_df" : train_df,
              "valid_df": valid_df,
              }

    torch.save(output, f"{OUTPUT_FOLDER}output-{model_name}.pt")
    
    base_dataset   = HotelTrainDataset(data_df, train_transform, data_path=IMAGE_FOLDER)
    base_loader    = DataLoader(base_dataset, num_workers=args.num_workers, batch_size=args.batch_size*4, shuffle=False)
    _, base_embeds = get_embeds(base_loader, model, "Generate embeddings for all images")
    data_df["embeddings"] = list(base_embeds)
    data_df.to_pickle(f"{OUTPUT_FOLDER}{model_name}_image-embeddings.pkl")

In [ ]:
%%time 

class args:
    epochs = 10
    lr = 1e-3
    batch_size = 16
    num_workers = 2  
    embed_size = 2048
    val_samples = 1
    backbone_name="efficientnet_b1"
    n_classes = data_df["hotel_id_code"].nunique()
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    continue_from_checkpoint = False

train_and_validate(args, data_df)